In [1]:
import torch
print(torch.__version__)
print(torch.cuda.get_device_name(0))
print("CUDA 是否可用：", torch.cuda.is_available())
print("PyTorch 使用的 CUDA 版本：", torch.version.cuda)



2.8.0.dev20250515+cu128
NVIDIA GeForce RTX 5070 Ti
CUDA 是否可用： True
PyTorch 使用的 CUDA 版本： 12.8


In [3]:
import ipywidgets as widgets
from IPython.display import display

slider = widgets.IntSlider(description='Test Slider')
display(slider)

IntSlider(value=0, description='Test Slider')

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)
from datasets import load_dataset
import torch

# 模型與 tokenizer
model_id = "meta-llama/Llama-2-7b-hf"
access_token = 
tokenizer = AutoTokenizer.from_pretrained(model_id, token=access_token)
tokenizer.pad_token = tokenizer.eos_token  # ✅ 這行是關鍵

# 量化設定（8bit）
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

# 載入模型
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=access_token,
    quantization_config=quantization_config,
    device_map="auto",
)

# 轉換為 LoRA 可訓練模型
model = prepare_model_for_kbit_training(model)

# LoRA 設定
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 測試生成文字
input_text = "Hello, world!"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_new_tokens=50,
        do_sample=True,
        top_p=0.95,
        temperature=0.8
    )
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("生成的文字：", output_text)

# 載入與處理資料集
dataset = load_dataset("mlabonne/guanaco-llama2-1k", split="train")

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 訓練參數
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=100,
    fp16=False,  # 如果你用的是支援 fp16 的 GPU，可設為 True
    logging_dir="./logs",
    report_to="tensorboard",
    remove_unused_columns=False  # ✅ 加上這個避免 text 欄位被自動移除
)

# Trainer 設定
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\bitsandbytes\autograd\_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


生成的文字： Hello, world! We’re the new folks.
This is our first post. We’re both new to the blogosphere.
Hopefully, we can post frequently, and be a bit more interactive than some blogs. We’ll


C:\Users\Cafe\AppData\Local\Temp\ipykernel_12356\1219571844.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\torch\_dynamo\eval_frame.py:879: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the di

Step,Training Loss
10,1.401500
20,1.408000
30,1.223000
40,1.272600
50,1.250200
60,1.244100
70,1.210300
80,1.274900
90,1.243200
100,1.569100


C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\peft\utils\other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6829e55c-0ffab87014b573a344b3a34c;0d01e35b-6b66-4d18-bd64-15a5aa3bf766)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-hf.
  warnings.warn(
C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\peft\utils\save_and_load.py:236: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-hf - will assume that the vocabulary was not modified.
  warnings.warn(
C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\torch\_dynamo\eval_frame.py:879: UserWarning: torch.utils.checkpoint: the use_reentrant paramet

TrainOutput(global_step=1500, training_loss=2.9113416430155437, metrics={'train_runtime': 1798.0956, 'train_samples_per_second': 1.668, 'train_steps_per_second': 0.834, 'total_flos': 6.0970588176384e+16, 'train_loss': 2.9113416430155437, 'epoch': 3.0})

In [5]:
# 儲存 LoRA adapter 權重
model.save_pretrained("lora_guanaco")
tokenizer.save_pretrained("lora_guanaco")


C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\peft\utils\other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6829ed11-453f87f819ec60590b89b609;ebb91fef-4761-43c1-8010-eed7f132548e)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-hf.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\torch\nn\modules\module.py:2425: UserWarning: for base_model.model.model.layers.27.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\torch\nn\modules\module.py:2425: UserWarning: for base_model.model.model.layers.27.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place

KeyError: 'base_model.model.model.lm_head'

In [2]:
import torch

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import os

os.makedirs("offload_dir", exist_ok=True)

try:
    print("🔧 開始設定基本參數...")
    base_model_id = "meta-llama/Llama-2-7b-hf"
    lora_path = "lora_guanaco"
    access_token = 
    print("⚙️ 設定量化參數...")
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_enable_fp32_cpu_offload=True
    )

    print("✅ 載入 tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_id, token=access_token, use_fast=False)
    tokenizer.pad_token = tokenizer.eos_token

    print("🧠 載入 base 模型...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        token=access_token,
        quantization_config=quantization_config,
        device_map="auto",
        offload_folder="offload_dir"
    )
    base_model.eval()
    print("✅ Base 模型載入完成")

    print("🧠 載入 LoRA 微調模型...")
    lora_model = PeftModel.from_pretrained(
        base_model,
        lora_path,
        device_map="auto",
        offload_folder="offload_dir",
        torch_dtype=torch.float16
    )
    lora_model.eval()
    print("✅ LoRA 微調模型載入完成")

except Exception as e:
    print("❌ 執行時發生錯誤：", e)


🔧 開始設定基本參數...
⚙️ 設定量化參數...
✅ 載入 tokenizer...
🧠 載入 base 模型...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Base 模型載入完成
🧠 載入 LoRA 微調模型...
✅ LoRA 微調模型載入完成


In [57]:

def generate_response(model, tokenizer, prompt: str, max_new_tokens=100):
    print("🧾 輸入 prompt:", prompt)
    
    # Tokenize prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.8,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text
def clean_generation(text, prompt):
    # 移除 prompt 本身
    text = text.replace(prompt, "")
    
    # 移除常見標籤
    for tag in ["[INST]", "[/INST]", "]", "["]:
        text = text.replace(tag, "")
    
    # 去除多餘空白與換行
    text = text.strip()
    
    return text

try:
    prompt = "Describe a peaceful sunset by the ocean."
    print("🧾 輸入 prompt:", prompt)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(base_model.device)
    print("🚀 開始用 Base 模型生成文字...")
    with torch.no_grad():
        base_output = base_model.generate(
            inputs,
            max_new_tokens=50,
            do_sample=True,
            top_p=0.95,
            temperature=0.8
        )
    print("✅ Base 模型生成完成")
    text = tokenizer.decode(base_output[0], skip_special_tokens=True)
    clean_text = clean_generation(text, prompt)
    
    print("📘 [Base 模型 輸出]:\n", clean_text)
    
    
    print("🚀 開始用 LoRA 微調模型生成文字...")
    with torch.no_grad():
        lora_output = lora_model.generate(
            inputs,
            max_new_tokens=50,
            do_sample=True,
            top_p=0.95,
            temperature=0.8
        )
    print("✅ LoRA 模型生成完成")
    text = tokenizer.decode(lora_output[0], skip_special_tokens=True)
    clean_text = clean_generation(text, prompt)
    print("📕 [LoRA 微調後模型 輸出]:\n", clean_text)

except Exception as e:
    print("❌ 執行時發生錯誤：", e)


🧾 輸入 prompt: Describe a peaceful sunset by the ocean.
🚀 開始用 Base 模型生成文字...
✅ Base 模型生成完成
📘 [Base 模型 輸出]:
 I am standing on a beach, surrounded by the serenity of a calm ocean     I am surrounded by the sounds of the
🚀 開始用 LoRA 微調模型生成文字...
✅ LoRA 模型生成完成
📕 [LoRA 微調後模型 輸出]:
 Sunset by the ocean is a beautiful and tranquil sight, with the sky and water shaded by a soft pink and orange color, and the gentle waves of the ocean crashing against


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

access_token = 
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=access_token,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    resume_download=True
)

prompt = "<s>[INST] <<SYS>>\nYou are a helpful assistant.\n<</SYS>>\nDescribe a peaceful sunset by the ocean in one sentence.[/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)


C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\transformers\models\auto\tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\transformers\models\auto\auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\Cafe\anaconda3\envs\Ai_fine_tuning\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [53]:
def clean_generation(text, prompt):
    # 移除 prompt 本身
    text = text.replace(prompt, "")
    
    # 移除常見標籤
    for tag in ["[INST]", "[/INST]", "]", "["]:
        text = text.replace(tag, "")
    
    # 去除多餘空白與換行
    text = text.strip()
    
    return text

prompt = "Describe a peaceful sunset by the ocean in one sentence."
inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)

output = base_model.generate(
    **inputs,
    max_new_tokens=30,
    do_sample=True,
    top_p=0.95,
    temperature=0.8,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

text = tokenizer.decode(output[0], skip_special_tokens=True)
clean_text = clean_generation(text, prompt)
print(clean_text)

" A serene sunset over a calm ocean, surrounded by a peaceful sea, a salty b


In [43]:
from datasets import load_dataset

dataset = load_dataset("mlabonne/guanaco-llama2-1k", split="train")

# 查看資料集大小
print(f"資料集大小: {len(dataset)}")

# 查看前幾筆資料
for i in range(3):
    print(dataset[i])


資料集大小: 1000
{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}
{'text': '<s>[INST] Самый великий человек из всех живших на планете? [/INST] Для начала нужно выбрать критерии вели

In [7]:
print(tokenizer.tokenize("<s>[INST] What is the meaning of life? [/INST]"))


['<s>', '[', 'INST', ']', '▁What', '▁is', '▁the', '▁meaning', '▁of', '▁life', '?', '▁[', '/', 'INST', ']']


In [8]:
print(lora_model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 